# SocioPatterns

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import igraph as ig
import dionysus as d

In [3]:
plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = 10, 6

In [4]:
%load_ext autoreload
%autoreload 2

## Data import

In [5]:
g = ig.read("data/sociopatterns/infectious/infectious.graphml")

In [6]:
g.summary()

'IGRAPH UN-- 10972 415912 -- \n+ attr: id (v), name (v), id (e), time (e)'

In [7]:
len(np.unique(g.es["time"]))

76944

In [ ]:
cliques = []
for t in np.unique(g.es["time"]):
    edges = g.es.select(time_eq=t)
    cliques.append(g.subgraph_edges(edges).maximal_cliques())

In [ ]:
maxcliques = g.maximal_cliques()

In [ ]:
def filt_subgraph(t):
    edges = g.es.select(time_eq=t)
    cliques = g.subgraph_edges(edges).maximal_cliques()
    return d.Filtration(cliques)

In [ ]:
times = np.sort(np.unique(g.es["time"]))

In [ ]:
import multiprocessing

In [ ]:
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
cliques = pool.map(clique_subgraph, times)

In [ ]:
pool.terminate()

In [ ]:
maxcliques = g.subgraph_edges(g.es.select(time_lt=times[30])).maximal_cliques()
filts = [filt_subgraph(t) for t in times[1:30]]

In [ ]:
maxfilt = d.Filtration(maxcliques)
for s in maxfilt:
    print(s)

In [ ]:
presences = [[s in filt for filt in filts] for s in maxfilt]

In [ ]:
presences[-1]

In [ ]:
[np.argwhere(np.array(p)[1:] ^ np.array(p)[:-1]).flatten().tolist() for p in presences]

## Cut into sliding windows

In [ ]:
duration = 0.1 * (times.max() - times.min())
duration * np.arange(10)

#### TODO overlap

In [8]:
def sliding_windows(g, res=0.1, overlap=0):
    times = np.array(g.es["time"])
    duration = res * (times.max() - times.min())
    windows = []
    for i in range(int(1/res)-1):
        edges = g.es.select(time_gt=times.min() + duration*i, time_lt=times.min() + duration*(i+1))
        windows.append(g.subgraph_edges(edges))
    return windows

In [9]:
wins = sliding_windows(g, 0.1)

In [10]:
for i in wins:
    print(max(i.es["time"]))

1241607579.0
1242302279.0
1242996839.0
1243691539.0
1244386179.0
1244995999.0
1245775339.0
1246468459.0
1247162039.0


## Zigzag persistence

Compute the maximal simplicial complex

In [11]:
def max_simplicial_complex(g):
    return d.Filtration(g.maximal_cliques())

In [12]:
def presence_times(g):
    max_simplicial_complex = d.Filtration(g.cliques())
    filts = []
    for t in np.sort(np.unique(g.es["time"])):
        edges = g.es.select(time_eq=t)
        cliques = g.subgraph_edges(edges).cliques()
        filts.append(d.Filtration(cliques))
    presences = [[s in filt for filt in filts] for s in max_simplicial_complex]
    presences = [np.argwhere(np.array(p)[1:] ^ np.array(p)[:-1]).flatten().tolist() for p in presences]
    return (max_simplicial_complex, presences)

In [13]:
(f, t) = presence_times(wins[2])

In [ ]:
zz, dgms, cells = d.zigzag_homology_persistence(f, t)

In [1]:
for s in d.Filtration([d.Simplex((0,1,2))]):
    print(s)

NameError: name 'd' is not defined